In [1]:
# accounts
import json
import solana
import time
import sys
import glob
from solana.rpc.api import Client
from solana.blockhash import Blockhash
from solana.publickey import PublicKey
from solana.system_program import CreateAccountWithSeedParams, create_account_with_seed, CreateAccountParams, create_account
from solana.transaction import Transaction, AccountMeta, TransactionInstruction
from solana.system_program import SYS_PROGRAM_ID
from spl.token.client import Token
from solana import keypair
from solana.publickey import PublicKey
from solana.sysvar import SYSVAR_RENT_PUBKEY

In [2]:
TOKEN_PROGRAM_ID: PublicKey = PublicKey("TokenkegQfeZyiNwAJbNbGKPFXCWuBvf9Ss623VQ5DA")

# first_key_in_key_folder = glob.glob("../keys/*")[0]
# payer_loaded_account = solana.keypair.Keypair(json.load(open(first_key_in_key_folder))[:32])
# payer_public_key = payer_loaded_account.public_key
payer_loaded_account = solana.keypair.Keypair.generate()
payer_public_key = payer_loaded_account.public_key



http_client = Client("https://api.devnet.solana.com")




# todo: change program_id to system program, change data size =0, change lampord for both alice and bob
alice_keypair = solana.keypair.Keypair.generate()
alice_pubkey = alice_keypair.public_key



bob_keypair = solana.keypair.Keypair.generate()
bob_pubkey = bob_keypair.public_key


print(f"Payer Account: {payer_public_key}")
print(f"alice Account: {alice_pubkey}")
print(f"bob Account: {bob_pubkey}")

Payer Account: 6vVPrx5MpNpsPudCYuGgeAwpjSZffohqXB2z4NnAk894
alice Account: iBgsen9yRWqTa11BqvNuEwSkEvu4c5g2DzMdGrXd3wL
bob Account: Eduww4vXpX593ghUzQSJnidRThFmACKjYPJ13nb1Cc6S


In [5]:
import time
# http_client.request_airdrop(payer_loaded_account.public_key, 1000000000)
# time.sleep(10)
http_client.request_airdrop(alice_pubkey, 1000000000)
time.sleep(15)
http_client.request_airdrop(bob_pubkey, 1000000000)
print(http_client.get_balance(alice_pubkey))
# print(http_client.get_balance(bob_pubkey))
# print(http_client.get_balance(payer_loaded_account.public_key))
# print(http_client.get_minimum_balance_for_rent_exemption(7)["result"])

{'jsonrpc': '2.0', 'result': {'context': {'slot': 86745718}, 'value': 0}, 'id': 6}


In [6]:
X_mint_account_address = Token.create_mint(conn= http_client, payer = payer_loaded_account, mint_authority = payer_loaded_account.public_key, decimals = 0, program_id = TOKEN_PROGRAM_ID, )
Y_mint_account_address = Token.create_mint(conn= http_client, payer = payer_loaded_account, mint_authority = payer_loaded_account.public_key, decimals = 0, program_id = TOKEN_PROGRAM_ID, )
alice_X_token_account = X_mint_account_address.create_associated_token_account(alice_pubkey)
alice_Y_token_account = Y_mint_account_address.create_associated_token_account(alice_pubkey)
bob_X_token_account = X_mint_account_address.create_associated_token_account(bob_pubkey)
bob_Y_token_account = Y_mint_account_address.create_associated_token_account(bob_pubkey)

In [115]:
X_mint_account_address.mint_to(alice_X_token_account,payer_loaded_account,1000)
Y_mint_account_address.mint_to(bob_Y_token_account,payer_loaded_account,100)

{'jsonrpc': '2.0',
 'result': '2X4FYkETEv7QgrSEPnkoheJJ5ygineKrwtnnVDMKgFX5sLg3peqWxhYwtsyQ1A7WcPandm6USmptX5rM4npnpSYq',
 'id': 102}

In [116]:
# escrow meta-data account
deployed_program_key = glob.glob("../target/deploy/*.json")[0]
deployed_program_key_account = solana.keypair.Keypair(json.load(open(deployed_program_key))[:32])
program_id = deployed_program_key_account.public_key

In [147]:
alice_Y_token_account

ABmUtVFoSJ7Rf35iCEEQ6mk5kejKcNMrdkDMus1PWKhd

In [135]:
# todo, create seed more specific
password = (10).to_bytes(8,byteorder='little') #b'password'
x_seeds = [
    b"vault_x",
    password,
    bytes(alice_pubkey),
    bytes(bob_pubkey),
    bytes(X_mint_account_address.pubkey),
    bytes(Y_mint_account_address.pubkey),
]

y_seeds = [
    b"vault_y",
    password,
    bytes(alice_pubkey),
    bytes(bob_pubkey),
    bytes(X_mint_account_address.pubkey),
    bytes(Y_mint_account_address.pubkey),
]

escrow_seeds = [
    b"escrow",
    password,
    bytes(alice_pubkey),
    bytes(bob_pubkey),
    bytes(X_mint_account_address.pubkey),
    bytes(Y_mint_account_address.pubkey),
]


vaultx, xseed = PublicKey.find_program_address(seeds=x_seeds,program_id=program_id)
vaulty, yseed = PublicKey.find_program_address(seeds=y_seeds,program_id=program_id)
escrow_address, escrow_seed = PublicKey.find_program_address(seeds=escrow_seeds,program_id=program_id)

In [141]:
vaulty

3yPETThCShhUPvRZGyzXVvzkSsXC8SFRUwtHQzt4dM3a

In [119]:
alice1 = [10, 140, 229, 196, 196, 173, 137, 112, 235, 104, 209, 194, 248, 49, 198, 12, 133, 186, 173, 133, 113, 136, 155, 72, 174, 44, 101, 204, 105, 117, 221, 119]
len(alice1)

32

In [120]:
bytes(alice_pubkey)

b'\n\x8c\xe5\xc4\xc4\xad\x89p\xebh\xd1\xc2\xf81\xc6\x0c\x85\xba\xad\x85q\x88\x9bH\xae,e\xcciu\xddw'

In [121]:
(229).to_bytes(1,byteorder='little')

b'\xe5'

In [137]:
print(vaultx)
print(vaulty)
print(escrow_address)
print(program_id)

B4kmMztf3YNhXTfbY6GGDV1hZc8Reh2SH3dpNx8h7ctw
3yPETThCShhUPvRZGyzXVvzkSsXC8SFRUwtHQzt4dM3a
A4NaZnruYPCqrno48QWTxTKCTZFXCcqTVvYbYS8qzad8
6rtZCTuF3hwKZhm6R6HJQGQ4nYq6ihYsCKXd4rUBw9KL


In [123]:
password = (16).to_bytes(8,byteorder='little')
data1 = pack('<BQQ', 0,13,15)  + password
data2 = pack('<BQQQ', 0,13,15,16)
print(data1)
print(data2)

b'\x00\r\x00\x00\x00\x00\x00\x00\x00\x0f\x00\x00\x00\x00\x00\x00\x00\x10\x00\x00\x00\x00\x00\x00\x00'
b'\x00\r\x00\x00\x00\x00\x00\x00\x00\x0f\x00\x00\x00\x00\x00\x00\x00\x10\x00\x00\x00\x00\x00\x00\x00'


In [124]:
pass1 = [16, 0, 0, 0, 0, 0, 0, 0]

In [125]:
(14).to_bytes(1,byteorder='little')

b'\x0e'

In [126]:
password

b'\x10\x00\x00\x00\x00\x00\x00\x00'

In [136]:
# initialize transaction
from struct import *
data = pack('<BQQ', 0,13,15)  + password

import struct
tx = Transaction()
tx_instruction = TransactionInstruction(
    keys=[AccountMeta(pubkey=escrow_address, is_signer=False, is_writable=True),
          AccountMeta(pubkey=X_mint_account_address.pubkey, is_signer=False, is_writable=False),
          AccountMeta(pubkey=Y_mint_account_address.pubkey, is_signer=False, is_writable=False),
          AccountMeta(pubkey=vaultx, is_signer=False, is_writable=True),
          AccountMeta(pubkey=vaulty, is_signer=False, is_writable=True),
          AccountMeta(pubkey=payer_public_key, is_signer=True, is_writable=False),
          AccountMeta(pubkey=alice_pubkey, is_signer=False, is_writable=False),
          AccountMeta(pubkey=bob_pubkey, is_signer=False, is_writable=False),
          AccountMeta(pubkey=TOKEN_PROGRAM_ID, is_signer=False, is_writable=False),
          AccountMeta(pubkey=SYSVAR_RENT_PUBKEY, is_signer=False, is_writable=False),
          AccountMeta(pubkey=SYS_PROGRAM_ID, is_signer=False, is_writable=False),
#           AccountMeta(pubkey=program_id, is_signer=False, is_writable=False),
         ],
    program_id=program_id,
    data=  data, #struct.pack("<LHB",1,1,1) # b58encode(bytes([9] * 7))#b'0001000100010001000100010001',
)

tx = tx.add(tx_instruction)


transaction_results = http_client.send_transaction(tx, *[payer_loaded_account])#, *[payer_loaded_account])

In [138]:
# time.sleep(0)
a = http_client.get_account_info(escrow_address)['result']['value']['data'][0]
import base64
print(len(base64.b64decode(a)))
print(unpack('<QQQ', base64.b64decode(a)[:24]))
print(unpack('<Q??', base64.b64decode(a)[208:218]))
print(base64.b64decode(a)[16:16+32])
bytes(alice_pubkey)

# Q 8 bytes
# a 400 bytes


# xval: u64,   8 byte
# yval: u64,   8 byte
# a_pub_key: Pubkey, 32 byte
# b_pub_key: Pubkey, 32 byte
# mint_x_pub_key: Pubkey,  32 byte
# mint_y_pub_key: Pubkey,  32 byte
# vault_x_pub_key: Pubkey, 32 byte
# vault_y_pub_key: Pubkey, 32 byte
# init_deposit_status: u64, 8 byte
# is_a_withdrawed: u8,    1byte
# is_b_withdrawed: u8,    1byte

218
(13, 15, 8109203664685861898)
(1, False, False)
b'\n\x8c\xe5\xc4\xc4\xad\x89p\xebh\xd1\xc2\xf81\xc6\x0c\x85\xba\xad\x85q\x88\x9bH\xae,e\xcciu\xddw'


b'\n\x8c\xe5\xc4\xc4\xad\x89p\xebh\xd1\xc2\xf81\xc6\x0c\x85\xba\xad\x85q\x88\x9bH\xae,e\xcciu\xddw'

In [107]:
base64.b64decode(a)[:227]

b'\r\x00\x00\x00\x00\x00\x00\x00\x0f\x00\x00\x00\x00\x00\x00\x00\x0e\x00\x00\x00\x00\x00\x00\x00\n\x8c\xe5\xc4\xc4\xad\x89p\xebh\xd1\xc2\xf81\xc6\x0c\x85\xba\xad\x85q\x88\x9bH\xae,e\xcciu\xddw\xca\x9c\xb4\xe7E\x82\xc7\xe5\xaa\xf4\xb6\x8a\x07XA*\x8b\x00\xaf@\xa5G\xff\xd3\xf5\x91p>\xfb\xf4\xe4_\x81\xa5\xd7\xe7\xdf\xc2\xb5Y\xb3ZUx\x8av7\x9b\xf3\x07pf\x92\x00\x04\x0c\xa2\xa7\xe8\x83b\xc5\xd8\xca\xb9\x94\xce,\x00e\xf3QF\x8c\xa2^\x13\x1d\xd4\r\xc0">@\xc9R\xe6\xbb\x80\x82a\x06t\xb1N\xd2(\x9d\x1a\xfa\xce\x97\x83\xfd\xeb\xb6\xa1\x88\xc3\n\x81\xab\x9e\x87Rc\\d\xef\xa5\xe6\xf86)%mo\x90|\xaf\x19xHY\x05%`\'\x83\xb9@\xcd~z0\x96\xde\xc1Y\xadv\n!\xea\x9d\xeb\xa5\x83\x001\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'

In [139]:
# deposite x
from struct import *
data = pack('<B', 1)

import struct
tx = Transaction()
tx_instruction = TransactionInstruction(
    keys=[AccountMeta(pubkey=escrow_address, is_signer=False, is_writable=True),
          AccountMeta(pubkey=alice_X_token_account, is_signer=False, is_writable=True), # x_a info
          AccountMeta(pubkey=vaultx, is_signer=False, is_writable=True),
          AccountMeta(pubkey=alice_pubkey, is_signer=True, is_writable=False),
          AccountMeta(pubkey=TOKEN_PROGRAM_ID, is_signer=False, is_writable=False),
         ],
    program_id=program_id,
    data=  data,
)

tx = tx.add(tx_instruction)


transaction_results = http_client.send_transaction(tx, *[alice_keypair])#, *[payer_loaded_account])

In [143]:
# deposite y
# time.sleep(30)
from struct import *
data = pack('<B', 1)

import struct
tx = Transaction()
tx_instruction = TransactionInstruction(
    keys=[AccountMeta(pubkey=escrow_address, is_signer=False, is_writable=True),
          AccountMeta(pubkey=bob_Y_token_account, is_signer=False, is_writable=True), # x_a info
          AccountMeta(pubkey=vaulty, is_signer=False, is_writable=True),
          AccountMeta(pubkey=bob_pubkey, is_signer=True, is_writable=False),
          AccountMeta(pubkey=TOKEN_PROGRAM_ID, is_signer=False, is_writable=False),
         ],
    program_id=program_id,
    data=  data,
)

tx = tx.add(tx_instruction)


transaction_results = http_client.send_transaction(tx, *[bob_keypair])#, *[payer_loaded_account])

In [146]:
# withdraw alice
from struct import *
data = pack('<B', 2) + password

import struct
tx = Transaction()
tx_instruction = TransactionInstruction(
    keys=[AccountMeta(pubkey=escrow_address, is_signer=False, is_writable=True),
          AccountMeta(pubkey=alice_Y_token_account, is_signer=False, is_writable=True), # x_a info
          AccountMeta(pubkey=vaulty, is_signer=False, is_writable=True),
          AccountMeta(pubkey=alice_pubkey, is_signer=True, is_writable=False),
          AccountMeta(pubkey=TOKEN_PROGRAM_ID, is_signer=False, is_writable=False),
         ],
    program_id=program_id,
    data=  data,
)

tx = tx.add(tx_instruction)


transaction_results = http_client.send_transaction(tx, *[alice_keypair])#, *[payer_loaded_account])

In [145]:
# withdraw bob
from struct import *
data = pack('<B', 2)+ password

import struct
tx = Transaction()
tx_instruction = TransactionInstruction(
    keys=[AccountMeta(pubkey=escrow_address, is_signer=False, is_writable=True),
          AccountMeta(pubkey=bob_X_token_account, is_signer=False, is_writable=True), # x_a info
          AccountMeta(pubkey=vaultx, is_signer=False, is_writable=True),
          AccountMeta(pubkey=bob_pubkey, is_signer=True, is_writable=False),
          AccountMeta(pubkey=TOKEN_PROGRAM_ID, is_signer=False, is_writable=False),
         ],
    program_id=program_id,
    data=  data,
)

tx = tx.add(tx_instruction)


transaction_results = http_client.send_transaction(tx, *[bob_keypair])#, *[payer_loaded_account])

RPCException: {'code': -32002, 'message': 'Transaction simulation failed: Error processing Instruction 0: Failed to serialize or deserialize account data: Unknown', 'data': {'accounts': None, 'err': {'InstructionError': [0, {'BorshIoError': 'Unknown'}]}, 'logs': ['Program 6rtZCTuF3hwKZhm6R6HJQGQ4nYq6ihYsCKXd4rUBw9KL invoke [1]', 'Program 6rtZCTuF3hwKZhm6R6HJQGQ4nYq6ihYsCKXd4rUBw9KL consumed 2401 of 200000 compute units', 'Program 6rtZCTuF3hwKZhm6R6HJQGQ4nYq6ihYsCKXd4rUBw9KL failed: Failed to serialize or deserialize account data: Unknown']}}

In [142]:
# Trouble shooting
# {'code': -32602, 'message': 'invalid transaction: index out of bounds'}: probabily signed with different account in send_transaction function
# debit credit, pobably you need to airdrop solana to the account
# Cross-program invocation with unauthorized signer or writable account: the signer has not signed correctly in send_transaction
# Failed to serialize or deserialize account data it was where I wanted to write from account data to a struct and account data length was more than strust length (remember struct length shuld be in bytes not bits)